### Chains

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

### Simple Chain(기본 체인)

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser   # 출력 파서(문자열로 변환)

# Prompt Template 생성
prompt = PromptTemplate(
    template="{country}의 수도는 어디인가요?",
    input_variables=['country']  # 사용자가 입력할 변수명 지정
)

# LLM Model 생성
model = ChatOpenAI(
    model="gpt-5-nano",
    temperature=0
    
 )

# OutputParser 생성
output_parser = StrOutputParser() # 모델 출력 결과에서 텍스트 내용(content)만 뽑아주는 파서 

# Chain 연결 ( 파이프(|) 연산자 사용)
chain = prompt | model | output_parser

result = chain.invoke(input={"country" : '대한민국'})

print(result)

서울특별시(서울)입니다.


### Sequential Chain(순차 체인)

In [9]:
# 체인 1: 번역

from langchain_openai import ChatOpenAI

# 모델생성
llm = ChatOpenAI(model='gpt-5-nano', temperature=0)

input_text = """
Slow down, you crazy child You're so ambitious for a juvenile But then if you're so smart, tell me Why are you still so afraid?
Where's the light? What's the hurry? You better cool it off before you burn it out You got so much to do and only So many hours in a day

"""

trans_prompt = PromptTemplate(
    template = "다음의 문장을 한글로 번역하세요 : {text}",
    input_variables=['text']
)

output_parser = StrOutputParser()
translation_chain = trans_prompt | llm | output_parser

trans_output = translation_chain.invoke(input_text)
print(trans_output)

천천히 가라, 너 참 미친 아이구나
너는 어릴 나이에 비해 너무 야망이 크구나
그런데 네가 그렇게 똑똑하다면 말해 보아라
왜 아직도 그렇게 두려운 거니?
빛은 어디에 있니? 서두르는 거니?
타버리기 전에 식히는 편이 낫다
해야 할 일이 너무 많고
하루에 주어진 시간은 그리 많지 않다


In [11]:
# 체인 2 : 요약 및 연결
from langchain_core.runnables import RunnableSequence # 여러 Runnable(체인)을 순서대로 실행할 수 있음

# 요약 프롬프트 생성
sum_prompt = PromptTemplate(
    template="다음의 글을 짧게 요약하세요 : {text}",
    input_variables=['text']
)

# 요약 체인 생성
summary_chain = sum_prompt | llm

# 체인 연결
# 앞의 체인의 출력이 뒤의 체인 입려으로 자동 전달된다. 
overall_chain = RunnableSequence(translation_chain, summary_chain)

final_output = overall_chain.invoke(input_text)
print(final_output)

content='야망이 크고 똑똑한 너에게 두려움을 이겨 빛을 찾아 천천히 가라고 조언하며, 해야 할 일이 많고 시간이 짧으니 서두르지 말고 차분히 식혀 두는 것이 낫다고 말한다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 645, 'prompt_tokens': 115, 'total_tokens': 760, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CqrdqG7IqAeAUx9U3xrvTxvliD4F7', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b5870-0e0c-7f41-9cef-ce35c370bac3-0' usage_metadata={'input_tokens': 115, 'output_tokens': 645, 'total_tokens': 760, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 576}}


In [12]:
# 조건부 체인
from langchain_core.runnables import RunnableBranch, RunnablePassthrough

llm = ChatOpenAI(model='gpt-5-nano', temperature=0)

# 평가 체인
grading_prompt = PromptTemplate(
    template="당신은 냉철한 평가자입니다. 아래 답변을 1~5점으로 평가해주세요:\n\n{text}",
    input_variables=['text']
)

grading_chain = grading_prompt | llm

# 기본 체인
default_prompt = PromptTemplate(
    template="당신은 사용자의 질문에 답하는 친절한 챗봇입니다.\n\n{text}",
    input_variables=['text']
)

default_chain = default_prompt | llm

def grading_routing_fn(input_dict) -> bool:
    # get(텍스트를 가져온다, 그런데 빈문자열일 때 프로그램이 꺼지는 것을 방지하기 위해서 ''을 넣는다. )
    text = input_dict.get('text', '')
    return text.strip().startswith('평가')

# 분기체인 
# RunnableBranch(
#   (조건함수, True일 때 실행할 체인),
#   False일 때 실행할 체인
# )

cond_chain = RunnableBranch(
    (grading_routing_fn, grading_chain),
    default_chain
)

# 알번 질문
print(cond_chain.invoke(input={'text' : '인공지능이 무엇인가요?'}))

content='인공지능(AI)은 컴퓨터가 인간처럼 지능적인 일을 하도록 만드는 기술과 연구 분야예요. 구체적으로는 데이터를 바탕으로 학습하고, 이해하고, 추론하며, 문제를 해결하고 의사결정을 하는 능력을 말합니다.\n\n주요 포인트\n- 무엇을 하나요: 패턴 인식(음성/이미지 인식), 자연어 이해, 학습과 적응, 계획과 의사결정 등 지능적 작업을 자동으로 수행합니다.\n- 두 가지 큰 범주\n  - 약한 AI(특정 작업에 특화된 시스템): 예를 들어 음성비서, 이미지 분류, 번역, 추천 시스템 등 특정 문제를 잘 해결하도록 만들어진 것들.\n  - 강한 AI(일반 지능): 인간처럼 모든 지적 과제를 스스로 해결하는 시스템. 현재 상용화된 건 아닙니다.\n- 핵심 기술\n  - 머신러닝: 데이터를 이용해 모델을 학습하고 예측하거나 판단하도록 하는 방법.\n  - 딥러닝: 인공신경망을 활용한 고급 머신러닝의 한 분류로, 이미지·음성·자연어 처리에 자주 쓰입니다.\n  - 학습 방법: 지도학습, 비지도학습, 강화학습 등.\n- 작동 원리(고수준)\n  1) 데이터 수집\n  2) 모델 설계(구조와 알고리즘 선택)\n  3) 학습(데이터로 가중치를 조정해 패턴을 학습)\n  4) 추론/예측(새로운 데이터에 대해 판단)\n- 예시\n  - 스마트폰 음성비서, 추천 알고리즘, 자율주행 자동차, 자동 번역, 이미지·의료 진단 보조 등\n- 한계와 주의점\n  - 데이터 편향에 따른 차별, 해석 가능성 부족, 프라이버시 및 안전 문제, 잘못된 일반화 가능성.\n\n원하시면 특정 분야의 AI 예시를 더 자세히 설명해 드릴게요. 또 다른 질문이 있나요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 1429, 'prompt_tokens': 32, 'total_tokens': 1461, 'completion_tokens_details': {'accepted_p

In [13]:
# 평가 질문
evaluation_input='평가: 인공지능(AI)은 컴퓨터가 인간처럼 지능적인 일을 하도록 만드는 기술과 연구 분야예요. '
print(cond_chain.invoke(input={'text': evaluation_input}))

content='평가: 4점\n\n이 정의는 AI의 기본 아이디어를 잘 전달합니다. 다만 “인간처럼”이라는 표현이 다소 비유적이며, AI가 모든 인간 지능을 모방하는 것은 아니고 특정 지능적 작업에 초점을 맞춘다는 점을 더 명확히 하면 좋습니다. 더 정확한 대안으로는:\n- “AI는 컴퓨터가 인간의 지능에 준하는 방식으로 지능적 작업을 수행하도록 하는 기술과 연구 분야.”\n- “AI는 데이터를 학습하고 문제를 해결하는 컴퓨터 시스템의 개발을 다루는 기술과 학문.”' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 1103, 'prompt_tokens': 59, 'total_tokens': 1162, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 960, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CqtD1bXuy7f6QSgvIpHEMnBpMQcNy', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b58cc-0068-7790-bc72-d25051a4ba42-0' usage_metadata={'input_tokens': 59, 'output_tokens': 1103, 'total_tokens': 1162, 'input_token_details': {'audio': 0, 'cach

In [14]:
# RunnablePassthrough 활용
# {'text' : RunnablePassthrough()} : 들어온 문자열을 그대로 'text' 키의 값으로 할당
pass_chain = {'text' : RunnablePassthrough()} | cond_chain

print(pass_chain.invoke('평가 : 코끼리는 어류이다.'))

content='점수: 1점\n\n이유: 제시된 진술은 사실과 다릅니다. 코끼리는 포유류이고 어류가 아닙니다. (코끼리 = 포유류; 어류 = 물고기)' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 765, 'prompt_tokens': 40, 'total_tokens': 805, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 704, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CqtHCTgCARNQsIsNNw2GlpzJq5M1r', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b58cf-f445-7de0-b563-4e784cd82a51-0' usage_metadata={'input_tokens': 40, 'output_tokens': 765, 'total_tokens': 805, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 704}}
